#  Notebook 0.7: Building a Minimal Interpreter or Compiler

###  Goal:
Build a very simple interpreter to understand parsing, tokenization, evaluation, and control flow.

###  Section 1: What Is an Interpreter?

- Reads code line-by-line
- Parses the text into **tokens**
- Converts tokens into **expressions (AST)**
- Evaluates expressions to compute a result

We'll build a simple calculator that can compute expressions like `1 + 2 * (3 + 4)`.

###  Section 2: Tokenizer (Lexer)

Breaks input string into tokens (numbers, operators, parentheses).

In [1]:
import re

def tokenize(expr):
    tokens = re.findall(r'\d+|[()+\-*\/]', expr)
    return tokens

tokenize("2 + 3 * (4 - 1)")

['2', '+', '3', '*', '(', '4', '-', '1', ')']

###  Section 3: Recursive Parser (AST Builder)

Turns tokens into nested tree structure. We'll use a recursive descent parser.
- Handles operator precedence (`*`, `/` > `+`, `-`)
- Handles parentheses

In [2]:
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0

    def peek(self):
        return self.tokens[self.pos] if self.pos < len(self.tokens) else None

    def eat(self):
        self.pos += 1

    def parse_expr(self):
        left = self.parse_term()
        while self.peek() in ('+', '-'):
            op = self.peek()
            self.eat()
            right = self.parse_term()
            left = (op, left, right)
        return left

    def parse_term(self):
        left = self.parse_factor()
        while self.peek() in ('*', '/'):
            op = self.peek()
            self.eat()
            right = self.parse_factor()
            left = (op, left, right)
        return left

    def parse_factor(self):
        tok = self.peek()
        if tok == '(':
            self.eat()
            expr = self.parse_expr()
            self.eat()  # eat ')'
            return expr
        else:
            self.eat()
            return int(tok)

In [3]:
# Parse example
tokens = tokenize("2 + 3 * (4 - 1)")
tree = Parser(tokens).parse_expr()
tree

('+', 2, ('*', 3, ('-', 4, 1)))

### Section 4: Evaluator

Walks the AST tree and computes the result.

In [4]:
def evaluate(tree):
    if isinstance(tree, int):
        return tree
    op, left, right = tree
    if op == '+': return evaluate(left) + evaluate(right)
    if op == '-': return evaluate(left) - evaluate(right)
    if op == '*': return evaluate(left) * evaluate(right)
    if op == '/': return evaluate(left) / evaluate(right)
    raise ValueError(f"Unknown operator {op}")

In [5]:
# Evaluate result
evaluate(tree)

11

###  Section 5: Full Interpreter

Combine everything into a `run()` function:

In [6]:
def run(expr):
    tokens = tokenize(expr)
    tree = Parser(tokens).parse_expr()
    return evaluate(tree)

# Test
run("1 + 2 * (3 + 4)")

15

###  Summary

| Part       | Role                                      |
|------------|-------------------------------------------|
| Tokenizer  | Breaks input into numbers and operators   |
| Parser     | Builds abstract syntax tree (AST)         |
| Evaluator  | Walks AST and computes result             |
| Interpreter| Combines all to run input expressions     |

**We built a mini-language for arithmetic from scratch!** This is the core idea behind all interpreters and compilers.